In [ ]:
# Setup input parameters
from datetime import datetime as dt
dbutils.widgets.text('bg_loadtimestamp', '')
bg_loadtimestamp = dbutils.widgets.get('bg_loadtimestamp')
bg_loadtimestamp_str = bg_loadtimestamp
if not bg_loadtimestamp:
    bg_loadtimestamp = 'CAST(NULL AS Timestamp)'
else:
    bg_loadtimestamp = f"CAST('{bg_loadtimestamp}' AS Timestamp)"




In [ ]:
# Setup logging
logger = spark._jvm.org.apache.log4j.Logger.getLogger('com.bigenius-x.application')
def info(targetName, message):
    logger.info(f'{targetName}: {message}')
    print(f"{dt.now().strftime('%Y/%m/%d, %H:%M:%S')} - {targetName}: {message}")




In [ ]:
# FactCleanseAction: order_items_Fact Cleanse Action_1

try:

    operation_metrics_collection = {}
    spark.sql("""DROP TABLE IF EXISTS `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`bg_erroneousrows_order_items_dataflow1`""")

    spark.sql("""CREATE TABLE `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`bg_erroneousrows_order_items_dataflow1` (
         `bg_cleanse_id` Long NOT NULL
    )
    USING delta
    """
    )

    spark.sql("""DROP TABLE IF EXISTS `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`bg_excludedrows_order_items_dataflow1`""")

    spark.sql("""CREATE TABLE `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`bg_excludedrows_order_items_dataflow1` (
         `bg_cleanse_id` Long NOT NULL
    )
    USING delta
    """
    )

    result_df = spark.sql(f"""
    INSERT
    OVERWRITE `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`bg_erroneousrows_order_items_dataflow1` (
         `bg_cleanse_id`
    )
    SELECT
         `bg_source`.`bg_cleanse_id` AS `bg_cleanse_id`
    FROM `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`cls_f_order_items` AS `bg_source`
    LEFT OUTER JOIN (
        SELECT
             `order_id`
            ,`order_status`
            ,`order_total`
            ,1 AS `bg_businesskeyexist`
        FROM `{marc_databricks_initiative#core#database_name}`.`{marc_databricks_initiative#core#schema_name}`.`cor_en_ent_orders_result`
        UNION ALL
        SELECT
             `order_id`
            ,`order_status`
            ,`order_total`
            ,1 AS `bg_businesskeyexist`
        FROM `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`cls_en_ent_orders_result`
    ) AS `bg_target`
       ON ((`bg_source`.`fk_ent_orders_order_id` = `bg_target`.`order_id`))
      AND ((`bg_source`.`fk_ent_orders_order_status` = `bg_target`.`order_status`))
      AND ((`bg_source`.`fk_ent_orders_order_total` = `bg_target`.`order_total`))
    WHERE `bg_target`.`bg_businesskeyexist` IS NULL
    """)
    result_df = spark.sql(f"""
    INSERT
    INTO `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`cls_f_order_items_error` (
         `bg_errorcount`
        ,`bg_errordescription`
        ,`bg_loadtimestamp`
        ,`bg_sourcesystem`
        ,`bg_effectivetimestamp`
        ,`bg_cleanse_id`
        ,`fk_ent_orders_order_id`
        ,`fk_ent_orders_order_status`
        ,`fk_ent_orders_order_total`
        ,`fk_ent_customers_customer_id`
        ,`fk_ent_customers_email_address`
        ,`fk_ent_customers_full_name`
        ,`fk_ent_products_product_id`
        ,`fk_ent_products_product_name`
        ,`fk_ent_products_unit_price`
        ,`fk_ent_products_product_image`
        ,`fk_ent_stores_store_name`
        ,`fk_ent_stores_web_address`
        ,`fk_ent_stores_latitude`
        ,`fk_ent_stores_longitude`
        ,`fk_ent_stores_physical_address`
        ,`fk_ent_stores_store_id`
        ,`unit_price`
        ,`quantity`
    )
    SELECT
         `bg_source`.`bg_errorcount` AS `bg_errorcount`
        ,'Missing foreign key value for relation to "Ent_orders" set to "Unknown"' AS `bg_errordescription`
        ,`bg_source`.`bg_loadtimestamp` AS `bg_loadtimestamp`
        ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
        ,`bg_source`.`bg_effectivetimestamp` AS `bg_effectivetimestamp`
        ,`bg_source`.`bg_cleanse_id` AS `bg_cleanse_id`
        ,`bg_source`.`fk_ent_orders_order_id` AS `fk_ent_orders_order_id`
        ,`bg_source`.`fk_ent_orders_order_status` AS `fk_ent_orders_order_status`
        ,`bg_source`.`fk_ent_orders_order_total` AS `fk_ent_orders_order_total`
        ,`bg_source`.`fk_ent_customers_customer_id` AS `fk_ent_customers_customer_id`
        ,`bg_source`.`fk_ent_customers_email_address` AS `fk_ent_customers_email_address`
        ,`bg_source`.`fk_ent_customers_full_name` AS `fk_ent_customers_full_name`
        ,`bg_source`.`fk_ent_products_product_id` AS `fk_ent_products_product_id`
        ,`bg_source`.`fk_ent_products_product_name` AS `fk_ent_products_product_name`
        ,`bg_source`.`fk_ent_products_unit_price` AS `fk_ent_products_unit_price`
        ,`bg_source`.`fk_ent_products_product_image` AS `fk_ent_products_product_image`
        ,`bg_source`.`fk_ent_stores_store_name` AS `fk_ent_stores_store_name`
        ,`bg_source`.`fk_ent_stores_web_address` AS `fk_ent_stores_web_address`
        ,`bg_source`.`fk_ent_stores_latitude` AS `fk_ent_stores_latitude`
        ,`bg_source`.`fk_ent_stores_longitude` AS `fk_ent_stores_longitude`
        ,`bg_source`.`fk_ent_stores_physical_address` AS `fk_ent_stores_physical_address`
        ,`bg_source`.`fk_ent_stores_store_id` AS `fk_ent_stores_store_id`
        ,`bg_source`.`unit_price` AS `unit_price`
        ,`bg_source`.`quantity` AS `quantity`
    FROM `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`cls_f_order_items` AS `bg_source`
    JOIN `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`bg_erroneousrows_order_items_dataflow1` AS `bg_error`
       ON `bg_source`.`bg_cleanse_id` = `bg_error`.`bg_cleanse_id`
    """)
    RowCountInserted = result_df.select("num_inserted_rows").collect()[0][0]
    operation_metrics = result_df.toJSON().collect()
    operation_metrics_collection['errorinsertstatement_{marc_databricks_initiative#cleanse#database_name}_{marc_databricks_initiative#cleanse#schema_name}_cls_f_order_items_error'] = operation_metrics
    result_df = spark.sql(f"""
    INSERT
    OVERWRITE `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`bg_erroneousrows_order_items_dataflow1` (
         `bg_cleanse_id`
    )
    SELECT
         `bg_source`.`bg_cleanse_id` AS `bg_cleanse_id`
    FROM `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`cls_f_order_items` AS `bg_source`
    LEFT OUTER JOIN (
        SELECT
             `customer_id`
            ,`email_address`
            ,`full_name`
            ,1 AS `bg_businesskeyexist`
        FROM `{marc_databricks_initiative#core#database_name}`.`{marc_databricks_initiative#core#schema_name}`.`cor_en_ent_customers_result`
        UNION ALL
        SELECT
             `customer_id`
            ,`email_address`
            ,`full_name`
            ,1 AS `bg_businesskeyexist`
        FROM `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`cls_en_ent_customers_result`
    ) AS `bg_target`
       ON ((`bg_source`.`fk_ent_customers_customer_id` = `bg_target`.`customer_id`))
      AND ((`bg_source`.`fk_ent_customers_email_address` = `bg_target`.`email_address`))
      AND ((`bg_source`.`fk_ent_customers_full_name` = `bg_target`.`full_name`))
    WHERE `bg_target`.`bg_businesskeyexist` IS NULL
    """)
    result_df = spark.sql(f"""
    INSERT
    INTO `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`cls_f_order_items_error` (
         `bg_errorcount`
        ,`bg_errordescription`
        ,`bg_loadtimestamp`
        ,`bg_sourcesystem`
        ,`bg_effectivetimestamp`
        ,`bg_cleanse_id`
        ,`fk_ent_orders_order_id`
        ,`fk_ent_orders_order_status`
        ,`fk_ent_orders_order_total`
        ,`fk_ent_customers_customer_id`
        ,`fk_ent_customers_email_address`
        ,`fk_ent_customers_full_name`
        ,`fk_ent_products_product_id`
        ,`fk_ent_products_product_name`
        ,`fk_ent_products_unit_price`
        ,`fk_ent_products_product_image`
        ,`fk_ent_stores_store_name`
        ,`fk_ent_stores_web_address`
        ,`fk_ent_stores_latitude`
        ,`fk_ent_stores_longitude`
        ,`fk_ent_stores_physical_address`
        ,`fk_ent_stores_store_id`
        ,`unit_price`
        ,`quantity`
    )
    SELECT
         `bg_source`.`bg_errorcount` AS `bg_errorcount`
        ,'Missing foreign key value for relation to "Ent_customers" set to "Unknown"' AS `bg_errordescription`
        ,`bg_source`.`bg_loadtimestamp` AS `bg_loadtimestamp`
        ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
        ,`bg_source`.`bg_effectivetimestamp` AS `bg_effectivetimestamp`
        ,`bg_source`.`bg_cleanse_id` AS `bg_cleanse_id`
        ,`bg_source`.`fk_ent_orders_order_id` AS `fk_ent_orders_order_id`
        ,`bg_source`.`fk_ent_orders_order_status` AS `fk_ent_orders_order_status`
        ,`bg_source`.`fk_ent_orders_order_total` AS `fk_ent_orders_order_total`
        ,`bg_source`.`fk_ent_customers_customer_id` AS `fk_ent_customers_customer_id`
        ,`bg_source`.`fk_ent_customers_email_address` AS `fk_ent_customers_email_address`
        ,`bg_source`.`fk_ent_customers_full_name` AS `fk_ent_customers_full_name`
        ,`bg_source`.`fk_ent_products_product_id` AS `fk_ent_products_product_id`
        ,`bg_source`.`fk_ent_products_product_name` AS `fk_ent_products_product_name`
        ,`bg_source`.`fk_ent_products_unit_price` AS `fk_ent_products_unit_price`
        ,`bg_source`.`fk_ent_products_product_image` AS `fk_ent_products_product_image`
        ,`bg_source`.`fk_ent_stores_store_name` AS `fk_ent_stores_store_name`
        ,`bg_source`.`fk_ent_stores_web_address` AS `fk_ent_stores_web_address`
        ,`bg_source`.`fk_ent_stores_latitude` AS `fk_ent_stores_latitude`
        ,`bg_source`.`fk_ent_stores_longitude` AS `fk_ent_stores_longitude`
        ,`bg_source`.`fk_ent_stores_physical_address` AS `fk_ent_stores_physical_address`
        ,`bg_source`.`fk_ent_stores_store_id` AS `fk_ent_stores_store_id`
        ,`bg_source`.`unit_price` AS `unit_price`
        ,`bg_source`.`quantity` AS `quantity`
    FROM `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`cls_f_order_items` AS `bg_source`
    JOIN `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`bg_erroneousrows_order_items_dataflow1` AS `bg_error`
       ON `bg_source`.`bg_cleanse_id` = `bg_error`.`bg_cleanse_id`
    """)
    RowCountInserted = result_df.select("num_inserted_rows").collect()[0][0]
    operation_metrics = result_df.toJSON().collect()
    operation_metrics_collection['errorinsertstatement_{marc_databricks_initiative#cleanse#database_name}_{marc_databricks_initiative#cleanse#schema_name}_cls_f_order_items_error'] = operation_metrics
    result_df = spark.sql(f"""
    INSERT
    OVERWRITE `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`bg_erroneousrows_order_items_dataflow1` (
         `bg_cleanse_id`
    )
    SELECT
         `bg_source`.`bg_cleanse_id` AS `bg_cleanse_id`
    FROM `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`cls_f_order_items` AS `bg_source`
    LEFT OUTER JOIN (
        SELECT
             `product_id`
            ,`product_name`
            ,`unit_price`
            ,`product_image`
            ,1 AS `bg_businesskeyexist`
        FROM `{marc_databricks_initiative#core#database_name}`.`{marc_databricks_initiative#core#schema_name}`.`cor_en_ent_products_result`
        UNION ALL
        SELECT
             `product_id`
            ,`product_name`
            ,`unit_price`
            ,`product_image`
            ,1 AS `bg_businesskeyexist`
        FROM `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`cls_en_ent_products_result`
    ) AS `bg_target`
       ON ((`bg_source`.`fk_ent_products_product_id` = `bg_target`.`product_id`))
      AND ((`bg_source`.`fk_ent_products_product_name` = `bg_target`.`product_name`))
      AND ((`bg_source`.`fk_ent_products_unit_price` = `bg_target`.`unit_price`))
      AND ((`bg_source`.`fk_ent_products_product_image` = `bg_target`.`product_image`))
    WHERE `bg_target`.`bg_businesskeyexist` IS NULL
    """)
    result_df = spark.sql(f"""
    INSERT
    INTO `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`cls_f_order_items_error` (
         `bg_errorcount`
        ,`bg_errordescription`
        ,`bg_loadtimestamp`
        ,`bg_sourcesystem`
        ,`bg_effectivetimestamp`
        ,`bg_cleanse_id`
        ,`fk_ent_orders_order_id`
        ,`fk_ent_orders_order_status`
        ,`fk_ent_orders_order_total`
        ,`fk_ent_customers_customer_id`
        ,`fk_ent_customers_email_address`
        ,`fk_ent_customers_full_name`
        ,`fk_ent_products_product_id`
        ,`fk_ent_products_product_name`
        ,`fk_ent_products_unit_price`
        ,`fk_ent_products_product_image`
        ,`fk_ent_stores_store_name`
        ,`fk_ent_stores_web_address`
        ,`fk_ent_stores_latitude`
        ,`fk_ent_stores_longitude`
        ,`fk_ent_stores_physical_address`
        ,`fk_ent_stores_store_id`
        ,`unit_price`
        ,`quantity`
    )
    SELECT
         `bg_source`.`bg_errorcount` AS `bg_errorcount`
        ,'Missing foreign key value for relation to "Ent_products" set to "Unknown"' AS `bg_errordescription`
        ,`bg_source`.`bg_loadtimestamp` AS `bg_loadtimestamp`
        ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
        ,`bg_source`.`bg_effectivetimestamp` AS `bg_effectivetimestamp`
        ,`bg_source`.`bg_cleanse_id` AS `bg_cleanse_id`
        ,`bg_source`.`fk_ent_orders_order_id` AS `fk_ent_orders_order_id`
        ,`bg_source`.`fk_ent_orders_order_status` AS `fk_ent_orders_order_status`
        ,`bg_source`.`fk_ent_orders_order_total` AS `fk_ent_orders_order_total`
        ,`bg_source`.`fk_ent_customers_customer_id` AS `fk_ent_customers_customer_id`
        ,`bg_source`.`fk_ent_customers_email_address` AS `fk_ent_customers_email_address`
        ,`bg_source`.`fk_ent_customers_full_name` AS `fk_ent_customers_full_name`
        ,`bg_source`.`fk_ent_products_product_id` AS `fk_ent_products_product_id`
        ,`bg_source`.`fk_ent_products_product_name` AS `fk_ent_products_product_name`
        ,`bg_source`.`fk_ent_products_unit_price` AS `fk_ent_products_unit_price`
        ,`bg_source`.`fk_ent_products_product_image` AS `fk_ent_products_product_image`
        ,`bg_source`.`fk_ent_stores_store_name` AS `fk_ent_stores_store_name`
        ,`bg_source`.`fk_ent_stores_web_address` AS `fk_ent_stores_web_address`
        ,`bg_source`.`fk_ent_stores_latitude` AS `fk_ent_stores_latitude`
        ,`bg_source`.`fk_ent_stores_longitude` AS `fk_ent_stores_longitude`
        ,`bg_source`.`fk_ent_stores_physical_address` AS `fk_ent_stores_physical_address`
        ,`bg_source`.`fk_ent_stores_store_id` AS `fk_ent_stores_store_id`
        ,`bg_source`.`unit_price` AS `unit_price`
        ,`bg_source`.`quantity` AS `quantity`
    FROM `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`cls_f_order_items` AS `bg_source`
    JOIN `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`bg_erroneousrows_order_items_dataflow1` AS `bg_error`
       ON `bg_source`.`bg_cleanse_id` = `bg_error`.`bg_cleanse_id`
    """)
    RowCountInserted = result_df.select("num_inserted_rows").collect()[0][0]
    operation_metrics = result_df.toJSON().collect()
    operation_metrics_collection['errorinsertstatement_{marc_databricks_initiative#cleanse#database_name}_{marc_databricks_initiative#cleanse#schema_name}_cls_f_order_items_error'] = operation_metrics
    result_df = spark.sql(f"""
    INSERT
    OVERWRITE `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`bg_erroneousrows_order_items_dataflow1` (
         `bg_cleanse_id`
    )
    SELECT
         `bg_source`.`bg_cleanse_id` AS `bg_cleanse_id`
    FROM `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`cls_f_order_items` AS `bg_source`
    LEFT OUTER JOIN (
        SELECT
             `store_name`
            ,`web_address`
            ,`latitude`
            ,`longitude`
            ,`physical_address`
            ,`store_id`
            ,1 AS `bg_businesskeyexist`
        FROM `{marc_databricks_initiative#core#database_name}`.`{marc_databricks_initiative#core#schema_name}`.`cor_en_ent_stores_result`
        UNION ALL
        SELECT
             `store_name`
            ,`web_address`
            ,`latitude`
            ,`longitude`
            ,`physical_address`
            ,`store_id`
            ,1 AS `bg_businesskeyexist`
        FROM `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`cls_en_ent_stores_result`
    ) AS `bg_target`
       ON ((`bg_source`.`fk_ent_stores_store_name` = `bg_target`.`store_name`))
      AND ((`bg_source`.`fk_ent_stores_web_address` = `bg_target`.`web_address`))
      AND ((`bg_source`.`fk_ent_stores_latitude` = `bg_target`.`latitude`))
      AND ((`bg_source`.`fk_ent_stores_longitude` = `bg_target`.`longitude`))
      AND ((`bg_source`.`fk_ent_stores_physical_address` = `bg_target`.`physical_address`))
      AND ((`bg_source`.`fk_ent_stores_store_id` = `bg_target`.`store_id`))
    WHERE `bg_target`.`bg_businesskeyexist` IS NULL
    """)
    result_df = spark.sql(f"""
    INSERT
    INTO `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`cls_f_order_items_error` (
         `bg_errorcount`
        ,`bg_errordescription`
        ,`bg_loadtimestamp`
        ,`bg_sourcesystem`
        ,`bg_effectivetimestamp`
        ,`bg_cleanse_id`
        ,`fk_ent_orders_order_id`
        ,`fk_ent_orders_order_status`
        ,`fk_ent_orders_order_total`
        ,`fk_ent_customers_customer_id`
        ,`fk_ent_customers_email_address`
        ,`fk_ent_customers_full_name`
        ,`fk_ent_products_product_id`
        ,`fk_ent_products_product_name`
        ,`fk_ent_products_unit_price`
        ,`fk_ent_products_product_image`
        ,`fk_ent_stores_store_name`
        ,`fk_ent_stores_web_address`
        ,`fk_ent_stores_latitude`
        ,`fk_ent_stores_longitude`
        ,`fk_ent_stores_physical_address`
        ,`fk_ent_stores_store_id`
        ,`unit_price`
        ,`quantity`
    )
    SELECT
         `bg_source`.`bg_errorcount` AS `bg_errorcount`
        ,'Missing foreign key value for relation to "Ent_stores" set to "Unknown"' AS `bg_errordescription`
        ,`bg_source`.`bg_loadtimestamp` AS `bg_loadtimestamp`
        ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
        ,`bg_source`.`bg_effectivetimestamp` AS `bg_effectivetimestamp`
        ,`bg_source`.`bg_cleanse_id` AS `bg_cleanse_id`
        ,`bg_source`.`fk_ent_orders_order_id` AS `fk_ent_orders_order_id`
        ,`bg_source`.`fk_ent_orders_order_status` AS `fk_ent_orders_order_status`
        ,`bg_source`.`fk_ent_orders_order_total` AS `fk_ent_orders_order_total`
        ,`bg_source`.`fk_ent_customers_customer_id` AS `fk_ent_customers_customer_id`
        ,`bg_source`.`fk_ent_customers_email_address` AS `fk_ent_customers_email_address`
        ,`bg_source`.`fk_ent_customers_full_name` AS `fk_ent_customers_full_name`
        ,`bg_source`.`fk_ent_products_product_id` AS `fk_ent_products_product_id`
        ,`bg_source`.`fk_ent_products_product_name` AS `fk_ent_products_product_name`
        ,`bg_source`.`fk_ent_products_unit_price` AS `fk_ent_products_unit_price`
        ,`bg_source`.`fk_ent_products_product_image` AS `fk_ent_products_product_image`
        ,`bg_source`.`fk_ent_stores_store_name` AS `fk_ent_stores_store_name`
        ,`bg_source`.`fk_ent_stores_web_address` AS `fk_ent_stores_web_address`
        ,`bg_source`.`fk_ent_stores_latitude` AS `fk_ent_stores_latitude`
        ,`bg_source`.`fk_ent_stores_longitude` AS `fk_ent_stores_longitude`
        ,`bg_source`.`fk_ent_stores_physical_address` AS `fk_ent_stores_physical_address`
        ,`bg_source`.`fk_ent_stores_store_id` AS `fk_ent_stores_store_id`
        ,`bg_source`.`unit_price` AS `unit_price`
        ,`bg_source`.`quantity` AS `quantity`
    FROM `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`cls_f_order_items` AS `bg_source`
    JOIN `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`bg_erroneousrows_order_items_dataflow1` AS `bg_error`
       ON `bg_source`.`bg_cleanse_id` = `bg_error`.`bg_cleanse_id`
    """)
    RowCountInserted = result_df.select("num_inserted_rows").collect()[0][0]
    operation_metrics = result_df.toJSON().collect()
    operation_metrics_collection['errorinsertstatement_{marc_databricks_initiative#cleanse#database_name}_{marc_databricks_initiative#cleanse#schema_name}_cls_f_order_items_error'] = operation_metrics
    result_df = spark.sql(f"""
    MERGE
    INTO `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`cls_f_order_items` AS `bg_target`
    USING (
        SELECT
             `bg_cleanse_id`
            ,COUNT(*) AS `bg_errorcount`
        FROM `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`bg_excludedrows_order_items_dataflow1`
        GROUP BY
             `bg_cleanse_id`
    ) AS `bg_error`
       ON `bg_target`.`bg_cleanse_id` = `bg_error`.`bg_cleanse_id`
    WHEN MATCHED
    THEN
        UPDATE 
        SET
             `bg_errorcount` = `bg_error`.`bg_errorcount`
    """)
    RowCountInserted = result_df.select("num_inserted_rows").collect()[0][0]
    operation_metrics = result_df.toJSON().collect()
    operation_metrics_collection['updateerrorcounts_{marc_databricks_initiative#cleanse#database_name}_{marc_databricks_initiative#cleanse#schema_name}_cls_f_order_items'] = operation_metrics

except Exception as e:
    info('CLS_F_order_items_Action', e)
    raise



In [ ]:
dbutils.notebook.exit(operation_metrics_collection)
